### 1. 环境初始化与数据加载
首先，我们导入numpy,pandas计算库，并读取原始数据集，并展示数据集基础信息

In [ ]:
import numpy as np 
import pandas as pd 

data_path = 'source_data'
train_data = pd.read_csv(data_path +'/train.csv')
test_data = pd.read_csv(data_path +'/test.csv')
# 查看训练数据的信息
train_data.info()
#train_data.columns

可以看到训练集共 891 个样本，其中所包含属性如下
- PassengerId => 乘客ID
- Survived => 乘客是否存活
- Pclass => 乘客等级(1/2/3等舱位)
- Name => 乘客姓名
- Sex => 性别
- Age => 年龄:缺失了 177 个值 ,   缺失率19.8%
- SibSp => 堂兄弟/妹个数
- Parch => 父母与小孩个数
- Ticket => 船票信息
- Fare => 票价
- Cabin => 客舱号码：缺失了  687 个值， 缺失率77.1%
- Embarked => 登船港口：缺失了 2 个值 缺失率0.2%

In [ ]:
# 查看训练数据的统计信息
train_data.describe()

### 2.探索性数据分析 ：
在建模之前，先分析可能影响的因素。通过可视化手段分析:
性别和年龄是否影响逃生机会（Sex, Age）
仓位登记和票价是否影响逃生机会？（Pclass, Fare）
登船地点是否存在隐含的生存规律？（Embarked）
下面的图表将分别从分类特征和数值分布两个维度揭示这些规律。

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(18, 12)) 


# 1. 性别 (Sex) - 分析男性和女性中各自的比率
survival_by_sex = train_data.groupby('Sex')['Survived'].mean()
survival_by_sex.plot(kind='bar', ax=axes[0, 0], color=['skyblue'])
axes[0, 0].set_title('A. Survival Rate by Sex', fontsize=14)
axes[0, 0].set_ylabel('Survival Rate')
axes[0, 0].set_xlabel('Sex')
axes[0, 0].tick_params(axis='x', rotation=0)


# 2. 客舱等级 (Pclass) - 分析客舱等级与生存率的关系
survival_by_pclass = train_data.groupby('Pclass')['Survived'].mean()
survival_by_pclass.plot(kind='bar', ax=axes[0, 1], color=['skyblue'])
axes[0, 1].set_title('B. Survival Rate by Pclass', fontsize=14)
axes[0, 1].set_ylabel('Survival Rate')
axes[0, 1].set_xlabel('Pclass')
axes[0, 1].tick_params(axis='x', rotation=0)


# 3. 登船港口 (Embarked) - 分析登船港口与生存率的关系
survival_by_embarked = train_data.groupby('Embarked')['Survived'].mean()
survival_by_embarked.plot(kind='bar', ax=axes[0 , 2], color=['skyblue'])
axes[0, 2].set_title('C. Survival Rate by Embarked Port', fontsize=14)
axes[0, 2].set_ylabel('Survival Rate')
axes[0, 2].set_xlabel('Embarked Port')
axes[0, 2].tick_params(axis='x', rotation=0)


# 4. 年龄 (Age) 分布 - 分析年龄分布与生存率的关系
age_survived = train_data[train_data['Survived'] == 1]['Age'].dropna()
age_perished = train_data[train_data['Survived'] == 0]['Age'].dropna()
axes[1, 0].hist(age_survived, bins=30, density=True, alpha=0.6, label='Survived (1)')
axes[1, 0].hist(age_perished, bins=30, density=True, alpha=0.6, label='Perished (0)')
axes[1, 0].set_title('D. Age Distribution by Survival Status (Histogram)', fontsize=14)
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Density')
axes[1, 0].legend(title='Survived')


# 5. 票价 (Fare) 分布 - 分析票价分布与生存率的关系
fare_data = [train_data[train_data['Survived'] == 0]['Fare'].dropna(), 
             train_data[train_data['Survived'] == 1]['Fare'].dropna()]
axes[1, 1].boxplot(fare_data, patch_artist=True, tick_labels=['Perished (0)', 'Survived (1)'])
axes[1, 1].set_title('E. Fare Distribution by Survival Status (Box Plot)', fontsize=14)
axes[1, 1].set_ylabel('Fare')
axes[1, 1].set_xlabel('Survival Status')
axes[1, 1].set_ylim(0, 150)

# 删除多余的子图
fig.delaxes(axes[1, 2]) 

plt.show()

### 3. 特征工程：高级特征构造 
- Title ：从 Name 提取 Mr, Mrs, Miss, Master。这极大地帮助树模型划分年龄段和社会地位。
- FamilySize ：SibSp + Parch + 1。将原始数据中的 SibSp 和 Parch 合并为 FamilySize。树模型可以很容易找到 FamilySize > 4 存活率骤降的分割点。
- IsAlone ：基于 FamilySize 创建的二分类特征 (0/1)。

In [ ]:
# 这里仅构造家庭大小为新特征
train_data["FamilySize"] = train_data["SibSp"] + train_data["Parch"] + 1
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"] + 1
print("创建完成")

### 4.高级数据清洗：基于随机森林的 MICE 填补
数据属性中较少的缺失值采用简单的均值/中位数/众数填充
数据属性中缺失值较多的采用 Iterative Imputer技术。 简单来说，将“预测年龄”视为一个回归问题：利用 Pclass, Fare, FamilySize 等已知信息，通过随机森林回归模型来精准反推缺失的 Age。

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # 开启实验性功能
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor


# 1. 先处理 Embarked (0.2% 缺失) - 只有 2 个缺失，直接mode众数填充
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

features_for_impute = ['Pclass', 'Sex', 'FamilySize', 'Fare', 'Age', 'Embarked']
df_impute = train_data[features_for_impute].copy()


# 2.独热编码：把所有特征数字化
# 'Sex' -> 0 / 1
df_impute['Sex'] = df_impute['Sex'].map({'male': 0, 'female': 1})
# 'Embarked' -> 0 / 1 / 2
df_impute['Embarked'] = df_impute['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}) 


# 3. 配置填充器：IterativeImputer
# estimator: 采用“随机森林回归”来预测缺失值
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(
        n_jobs=-1,              # n_jobs=-1 使用所有可用的 CPU 核心来加速训练
        random_state=42
    ),
    max_iter=10,                # max_iter: 迭代轮数10次
    random_state=42             # 设置随机种子，使得每次运行结果相同
)

# 4. 训练阶段 (Fit) - 此时只看不做
# 这一步，rf_imputer 只是在“观察”数据，寻找年龄和其他特征的规律
print("正在学习训练集的规律...")
rf_imputer.fit(df_impute) 

# 5. 应用阶段 (Transform) - 此时只做不想
# 这一步，用刚才学到的规律，把训练集自己的坑填上
print("正在填充训练集...")
imputed_data = rf_imputer.transform(df_impute)



# 6. 还原回 DataFrame
df_filled = pd.DataFrame(imputed_data, columns=features_for_impute)

# 把填好的 Age 塞回原始数据
train_data['Age'] = df_filled['Age'].round(0)

# 7. 修正非法值
# 如果预测出负数，强制变为 1；如果太大，限制在 80
train_data.loc[train_data['Age'] < 0, 'Age'] = 1
train_data.loc[train_data['Age'] > 80, 'Age'] = 80

print("基于随机森林的智能填充完成！")
print(train_data[['Age', 'Embarked']].isnull().sum()) # 检查缺失值数量
print("Age 缺失值填充完毕!前10个结果预览:")
print(train_data['Age'].head(10))

### 5.测试集填补 

In [ ]:
# 修复 Test 集的 缺失值

# 1. 修复 Fare (1 个缺失值) - 用 Train 的中位数去填，假设他买了张普通票
test_data['Fare'] = test_data['Fare'].fillna(train_data['Fare'].median())

# 2. 修复 Age (86 个缺失值)
# 用训练好的 rf_imputer 去预测 Test 的年龄
# 第一步：准备数据 (格式必须和训练 imputer 时一模一样)
features_for_impute = ['Pclass', 'Sex', 'FamilySize', 'Fare', 'Age', 'Embarked']
df_test_impute = test_data[features_for_impute].copy()

# 3.独热编码 - 特征数字化 (Sex/Embarked 转数字)
df_test_impute['Sex'] = df_test_impute['Sex'].map({'male': 0, 'female': 1})
df_test_impute['Embarked'] = df_test_impute['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# 4.应用 Train训练集 学到的规律 (Transform)，只需要 transform
test_imputed_data = rf_imputer.transform(df_test_impute)

# 5.把填好的 Age 填充 test_data
df_test_filled = pd.DataFrame(test_imputed_data, columns=features_for_impute)
test_data['Age'] = df_test_filled['Age'].round(0).astype(int)

# 6.修正非法值
test_data.loc[test_data['Age'] < 0, 'Age'] = 1
test_data.loc[test_data['Age'] > 80, 'Age'] = 80


print("现在剩余的 NaN 情况：")
print(test_data[['Age', 'Fare']].isnull().sum()) # 查看缺失值数量



### 6.模型训练与提交
我们使用 随机森林分类器对test集进行预测

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "Embarked", "FamilySize", "Fare"]
# 独热编码
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# 自动补齐测试集中缺失的列（补为0），并确保顺序一致
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# 模型训练与预测 
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

### 7.模型评估：交叉验证 (CV)
采用交叉验证 (Cross Validation) 分析模型的表现。通过将训练数据切分为 5 份，轮流进行“模拟考试”，得到一个比单一分数更可信的平均分和标准差，从而判断模型的泛化能力。

In [ ]:
from sklearn.model_selection import cross_val_score
# cv=5 表示考 5 次，每次考题都不一样
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"每次考试的分数: {scores}")
print(f"本地估算平均分: {scores.mean():.4f}")
print(f"标准差 (越小越稳): {scores.std():.4f}")